<a href="https://colab.research.google.com/github/STRIDES-Codes/NL4Cell-Integrating-NLP-with-single-cell-data-analysis/blob/main/CellAttention_with_RoBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# %%capture
!pip install git+https://github.com/justinphan3110/transformers.git

  Cloning https://github.com/justinphan3110/transformers.git to /tmp/pip-req-build-uy34wbk1
  Running command git clone -q https://github.com/justinphan3110/transformers.git /tmp/pip-req-build-uy34wbk1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 7.2 MB/s 
     |████████████████████████████████| 3.3 MB 61.0 MB/s 
  Created wheel for transformers: filename=transformers-4.9.0.dev0-py3-none-any.whl size=2534653 sha256=65501d0658c08f09a7ad0859ab2340b073d6374460156294606fc5f3ccfce7b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-_1p8ryf5/wheels/18/e4/e5/081c522c96f60fe34d3487499b8415b7c13a42d319b2e4a7af
Successfully built transformers


In [ ]:
from transformers import EncoderDecoderModel, RobertaTokenizer, RobertaConfig, RobertaModel
import torch
import numpy as np

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
  print(torch.cuda.get_device_name())
else:
  device = torch.device('cpu')
  

Tesla T4


## Processed Data

In [ ]:
!gsutil cp gs://cytereader/preprocessed_cell_corpus_test.pkl.zip .
!unzip preprocessed_cell_corpus_test.pkl.zip

Copying gs://cytereader/preprocessed_cell_corpus_test.pkl.zip...
/ [1 files][218.2 MiB/218.2 MiB]                                                
Operation completed over 1 objects/218.2 MiB.                                    
Archive:  preprocessed_cell_corpus_test.pkl.zip
  inflating: preprocessed_cell_corpus_test.pkl  


In [ ]:
import numpy as np


data = np.load('sample_sc_data.pkl', allow_pickle=True)
data.head()

,CD45,CD196_CCR6,CD181_CXCR1,HLA_DR,CD15,CD31_PECAM1,CD8a,CD182_CXCR2,IgA,CD66ace,CD63,CD14,CD66b,CD62L_Lselectin,CD3,CD27,CD86,CD10,CD197_CCR7,CD28,CD11c,CD33,CD161,CD45RO,CD24,CD38,CD278_ICOS,CD32,CD152_CTLA4,IgM,CD184_CXCR4,CD279_PD1,CD56,CD16
0,-0.279450,-0.707366,0.384317,0.836923,1.096945,0.228808,-0.096544,0.767240,0.820186,0.665852,0.272751,4.321285,1.569835,0.634354,-0.550853,2.161058,-0.459699,0.742737,-0.778183,0.656047,0.445491,1.299726,1.210270,1.080307,1.435914,-0.992491,2.013223,0.410796,-0.387851,-0.697368,-0.624541,1.979515,-0.26531,0.064496
3,0.659602,0.427319,1.279784,1.255604,1.520442,1.190339,0.864978,0.910992,1.138400,0.634851,-0.124700,-0.579862,1.295523,-0.108997,0.050769,-0.497900,0.743166,1.184591,0.924837,-0.786118,0.072120,1.154462,-0.744500,1.074286,1.596360,-0.873943,0.947234,0.238758,0.357634,-0.306862,1.060050,-0.236844,-0.26531,0.546826
5,-1.396816,-0.485352,0.625359,0.504764,-0.261098,-0.352752,1.417640,0.616807,0.402000,-0.639013,-0.381911,0.178510,-0.597169,0.566762,0.631586,-0.497900,-0.459699,0.237092,0.345890,-0.786118,-0.630285,0.776386,0.744373,-0.374436,0.329600,-0.386987,-0.416999,0.112814,-0.387851,-0.697368,0.031788,-0.732879,-0.26531,0.862344
6,1.799861,1.150422,0.654544,1.012722,0.363497,1.149256,1.256522,0.459767,1.019238,-0.291694,2.270498,1.735677,0.931907,1.802715,-0.550853,-0.497900,-0.459699,-3.741549,-0.778183,-0.786118,1.101045,1.279933,0.807132,1.823290,1.543040,0.673735,-0.416999,0.917741,0.065323,-0.697368,1.549627,-0.732879,-0.26531,-3.492455
7,1.166063,-0.277376,0.547420,0.546770,-0.232914,-1.405149,-0.732767,0.166671,-0.327599,0.354038,-0.319064,0.331496,1.022024,-1.623367,-0.550853,0.031693,0.827035,0.173805,-0.778183,-0.786118,0.707982,0.069510,-1.401472,0.077694,0.781131,1.022201,-0.416999,0.666394,-0.387851,-0.697368,1.976239,-0.732879,-0.26531,0.418635


In [ ]:
data.shape

(100, 34)

In [ ]:
map = {}

for protein in data:
  for value in data[protein]:
    if protein not in map:
      map[protein] = set()
    map[protein].add(value)
    if len(map[protein]) == 2:
      break


In [ ]:
l = []

for key in map:
  l.append(map[key])
l

[{-0.27945008873939514, 0.6596015691757202},
 {-0.707366406917572, 0.42731931805610657},
 {0.384317010641098, 1.2797837257385254},
 {0.8369234204292297, 1.2556043863296509},
 {1.0969451665878296, 1.5204423666000366},
 {0.2288081794977188, 1.1903390884399414},
 {-0.09654375910758972, 0.8649781346321106},
 {0.7672399282455444, 0.9109917283058167},
 {0.8201862573623657, 1.1384004354476929},
 {0.6348510384559631, 0.6658516526222229},
 {-0.12469985336065292, 0.272750586271286},
 {-0.5798620581626892, 4.321284770965576},
 {1.2955232858657837, 1.569834589958191},
 {-0.10899665951728821, 0.6343538165092468},
 {-0.5508525371551514, 0.050768833607435226},
 {-0.49790024757385254, 2.161057710647583},
 {-0.4596991240978241, 0.7431658506393433},
 {0.7427371740341187, 1.1845905780792236},
 {-0.7781826257705688, 0.9248367547988892},
 {-0.7861180305480957, 0.6560471057891846},
 {0.07212047278881073, 0.4454909563064575},
 {1.1544617414474487, 1.2997262477874756},
 {-0.7445002198219299, 1.210269689559936

In [ ]:
def getProtein(value):
  for key in map:
    if value in map[key]:
      return key

def getSuffix(protein, value):
  if max(map[protein]) - value == 0:
    return '+'
  else:
    return '-'

In [ ]:
def split_dataframe(df, chunk_size = 1000000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

data = split_dataframe(data)

In [ ]:
data

[         CD45  CD196_CCR6  CD181_CXCR1  ...  CD279_PD1      CD56      CD16
 0   -0.279450   -0.707366     0.384317  ...   1.979515 -0.265310  0.064496
 3    0.659602    0.427319     1.279784  ...  -0.236844 -0.265310  0.546826
 5   -1.396816   -0.485352     0.625359  ...  -0.732879 -0.265310  0.862344
 6    1.799861    1.150422     0.654544  ...  -0.732879 -0.265310 -3.492455
 7    1.166063   -0.277376     0.547420  ...  -0.732879 -0.265310  0.418635
 ..        ...         ...          ...  ...        ...       ...       ...
 128  0.516790   -0.707366    -0.021224  ...  -0.732879 -0.265310  0.114201
 129  0.211735   -0.707366     1.809287  ...  -0.732879 -0.265310 -1.042863
 130 -1.303168   -0.707366    -1.153692  ...  -0.732879 -0.265310  0.306272
 131 -0.393257    0.144885    -1.153692  ...  -0.732879 -0.265310 -0.168588
 133  0.451776   -0.707366     0.861812  ...  -0.732879  2.955293  0.375529
 
 [100 rows x 34 columns]]

In [ ]:
def getText( x ):
    protein_names = []
    for i,protein_value in enumerate(x):
      protein = getProtein(protein_value)
      suffix = getSuffix(protein, protein_value)

      protein_names.append(protein + suffix)

    return ' '.join(protein_names)

for index,chunk in enumerate(data):
  text_representation = np.apply_along_axis( getText, axis=1, arr=chunk)

  with open(f'preprocessed_cell_corpus_{index}.txt', 'w') as file:
    for line in text_representation:
      file.write(f'{line}\n')
  print(f'done chunk {index}')


KeyError: ignored

In [ ]:
text_representation

NameError: ignored

In [ ]:
text_representation

NameError: ignored

## Train

In [ ]:
!pip install tokenizers

In [ ]:
!gsutil cp gs://cytereader/preprocessed_cell_corpus_0.txt . 
!gsutil cp gs://cytereader/preprocessed_cell_corpus_1.txt . 

Copying gs://cytereader/preprocessed_cell_corpus_0.txt...
\ [1 files][267.0 MiB/267.0 MiB]                                                
Operation completed over 1 objects/267.0 MiB.                                    
Copying gs://cytereader/preprocessed_cell_corpus_1.txt...
\ [1 files][267.0 MiB/267.0 MiB]                                                
Operation completed over 1 objects/267.0 MiB.                                    


In [ ]:
from tokenizers import BertWordPieceTokenizer

wb_tokenizer = BertWordPieceTokenizer(clean_text=True,
                                      strip_accents=True, lowercase=False)

wb_tokenizer.train(['preprocessed_cell_corpus_0.txt', 'preprocessed_cell_corpus_1.txt'],
                   vocab_size=10000, min_frequency=2,
                   special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])
wb_tokenizer.save_model(".")

['./vocab.txt']

In [42]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaModel, RobertaForMaskedLM

tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
configuration = RobertaConfig(vocab_size=1000)


model = RobertaForMaskedLM(configuration)

loading file https://huggingface.co/roberta-large/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/7c1ba2435b05451bc3b4da073c8dec9630b22024a65f6c41053caccf2880eb8f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-large/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/20b5a00a80e27ae9accbe25672aba42ad2d4d4cb2c4b9359b50ca8e34e107d6d.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-large/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-large/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/roberta-large/resolve/main/tokenizer_config.json from cache at None
loading file https://huggingface.co/roberta-large/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/e16a2590deb9e6d73711d6e05bf27d832fa8c1162d807222

In [ ]:
model.num_parameters()

86811880

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="preprocessed_cell_corpus_0.txt",
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:124: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Creating features from dataset file at preprocessed_cell_corpus_0.txt


CPU times: user 10min 25s, sys: 23.3 s, total: 10min 48s
Wall time: 10min 48s


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./cellAttention",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# %%time
trainer.train()

***** Running training *****
  Num examples = 1000000
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 15625


TypeError: ignored

In [ ]:
trainer.save_model("./cellAttention")

Saving model checkpoint to ./cellAttention
Configuration saved in ./cellAttention/config.json
Model weights saved in ./cellAttention/pytorch_model.bin


In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./cellAttention",
    tokenizer="./cellAttention"
)

loading configuration file ./cellAttention/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.9.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 1000
}

loading configuration file ./cellAttention/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden

OSError: ignored

In [ ]:
fill_mask("CD45+ CD196_CCR6+ CD181_CXCR1- HLA_DR- CD15- CD31_PECAM1- CD8a- CD182_CXCR2[MASK] CD66ace- CD63- CD14- CD66b- CD62L_Lselectin- CD3+ CD27- CD86+ CD10- CD197_CCR7+ CD28- CD11c- CD33- CD161- CD45RO- CD24- CD38+ CD278_ICOS- CD32- CD152_CTLA4+ IgM+ CD184_CXCR4+ CD279_PD1- CD56+ CD16-")

In [ ]:
input = 'CD45+ CD196_CCR6+ CD181_CXCR1- HLA_DR- CD15-'
masked_input = 'CD45+ CD196_CCR6+ CD181_CXCR1[MASKED] HLA_DR- CD15-'

output = 'CD45+ CD196_CCR6+ CD181_CXCR1-/+ HLA_DR- CD15-'